In [1]:
from Module import *
from Optimizer import *
from DataTools import *

In [2]:
train_dataset = DataSet(['train-images-idx3-ubyte', 'train-labels-idx1-ubyte'], data_type='mnist')
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


test_dataset = DataSet(['t10k-images-idx3-ubyte', 't10k-labels-idx1-ubyte'], data_type='mnist')
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [3]:
def ResidualMLP(input_dim, hidden_dim, output_dim, num_blocks):
    net = Sequential(Linear(input_dim, hidden_dim), *[Residual(Linear(hidden_dim, hidden_dim)) for _ in range(num_blocks)], Linear(hidden_dim, output_dim), ReLU())
    return net

In [4]:
net = ResidualMLP(784, 128, 1, 3)
optimizer = SGD(net.parameters(), lr=0.01)
scheduler = ConstantScheduler(optimizer=optimizer, lr=0.01)
loss_fn = MSELoss()

In [12]:
num_epochs = 10
for epoch in range(num_epochs):
    for data, labels in train_loader:
        data = data.reshape(data.shape[0], -1)
        data = Tensor.make_const(data)
        print(f"data shape: {type(data)}")
        optimizer.reset_grad()
        output = net(data)
        loss = loss_fn(output, labels.reshape(-1, 1))
        loss.backward()
        optimizer.step()
        scheduler.step()

data shape: <class 'core.Tensor'>


ValueError: operands could not be broadcast together with shapes (64,1) (128,1) 